## Initialize dataframes

In [1]:
from Startup import *

finished standard imports
[('spark.logConf', u'True'), ('spark.executor.cores', u'1'), ('spark.default.parallelism', u'10'), ('spark.cores.max', u'4'), ('spark.executor.memory', u'3g'), ('spark.app.name', u'Weather_PCA')]
started SparkContext and SQLContext in 13.16 seconds
loaded weather.parquet in 7.08 seconds
loaded stations.parquet in 0.17 seconds
registered dataframes as tables in 0.22 seconds


In [ ]:
%%time
print('df has %d rows, stations has %d rows'%(df.count(),stations.count()))

## Check how many rows corresponds to each measurement

In [2]:
%%time
query="""
SELECT measurement,COUNT(measurement) AS count
FROM weather  
GROUP BY measurement 
ORDER BY count
"""
print(query)
measurements_pdf=sqlContext.sql(query).toPandas()


SELECT measurement,COUNT(measurement) AS count
FROM weather  
GROUP BY measurement 
ORDER BY count

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 18.1 s


In [3]:
measurements_pdf.tail(20)

,measurement,count
113,WT14,38369
114,WT18,53072
115,WT08,53648
116,WESD,55740
117,WT16,73840
118,WT06,103479
119,WT11,120277
120,WT05,148289
121,WT04,163690
122,DWPR,180462


## The most populated measurements
* The most populated measurements are the ones which we have been looking at: `PRCP,TMIN,TMAX,SNWD,TOBS`
* After those the most common ones are **weather types**. These are binary features that describe the daily weather qualitatively.

## Weather-Type Fields (WT**)

(from `/mnt/workspace/WeatherInfo/ghcnd-readme.txt`)

           WT** = Weather Type where ** has one of the following values:

                  01 = Fog, ice fog, or freezing fog (may include heavy fog)
                  02 = Heavy fog or heaving freezing fog (not always
                       distinquished from fog)
                  03 = Thunder
                  04 = Ice pellets, sleet, snow pellets, or small hail
                  05 = Hail (may include small hail)
                  06 = Glaze or rime
                  07 = Dust, volcanic ash, blowing dust, blowing sand, or
                       blowing obstruction
                  08 = Smoke or haze
                  09 = Blowing or drifting snow
                  10 = Tornado, waterspout, or funnel cloud
                  11 = High or damaging winds
                  12 = Blowing spray
                  13 = Mist
                  14 = Drizzle
                  15 = Freezing drizzle
                  16 = Rain (may include freezing rain, drizzle, and
                       freezing drizzle)
                  17 = Freezing rain
                  18 = Snow, snow pellets, snow grains, or ice crystals
                  19 = Unknown source of precipitation
                  21 = Ground fog
                  22 = Ice fog or freezing fog

* *Possibe Question*: The categries have probably chosen by a human not by a formula. Can we model the join distribution of all of these ccategories? In other words, is catergory $i$ approximately a subset of category $j$? are they disjoint?

## MDPR, MWPR, DAPR

(from `/mnt/workspace/WeatherInfo/ghcnd-readme.txt`)

* **MDPR** = Multiday precipitation total (tenths of mm; use with DAPR and DWPR, if available)
* **DAPR** = Number of days included in the multiday precipiation total (MDPR)
* **DWPR** = Number of days with non-zero precipitation included in multiday precipitation total (MDPR)
                  
Taking the multiday total is similar to smoothing. A smoothed precipitation graph is easier to analyze using PCA.

## Visualizing weather types

In [ ]:
%%time
station='USW00093819'
query="""
SELECT *
FROM weather
WHERE measurement='WT01' and station='%s'
"""%station
print(query)
WT01_df=sqlContext.sql(query)
WT01_pdf=WT01_df.toPandas()

In [ ]:
WT01_pdf.head(4)

In [ ]:
%pylab inline

In [ ]:
wt01_graphs=[]
for i in range(WT01_pdf.shape[0]):
    graph=unpackArray(WT01_pdf.iloc[i,3],np.float16)
    graph=np.nan_to_num(graph)
    wt01_graphs.append(graph)
wt01_graphs=np.stack(wt01_graphs)

In [ ]:
from lib.YearPlotter import YearPlotter
W=wt01_graphs[:4,:]
fig, ax = plt.subplots(figsize=[10,10]);
YP=YearPlotter()
YP.plot(W.transpose(),fig,ax,title='WT01, %s'%(station));
#plt.savefig('percipitation.png')
#title('A sample of graphs');